In [1]:
%load_ext Cython

In [2]:
from math import log, exp, sqrt, pow, erf
from scipy.stats import norm

In [3]:
def norm_cdf(x):
    return 0.5*(1.0+erf(x/sqrt(2.0)))

def BS_price_1(S, K, T, vol, r, q, typ):
    """return BS price for call/put for typ +1/-1"""
    d1 = (log(S/K)+(r-q+0.5*pow(vol, 2))*T)/(vol*sqrt(T))
    d2 = d1-vol*sqrt(T)
    price = typ*S*exp(-q*T)*norm.cdf(typ*d1)-typ*K*exp(-r*T)*norm.cdf(typ*d2)
    return price

def BS_price_2(S, K, T, vol, r, q, typ):
    """return BS price for call/put for typ +1/-1"""
    d1 = (log(S/K)+(r-q+0.5*pow(vol, 2))*T)/(vol*sqrt(T))
    d2 = d1-vol*sqrt(T)
    price = typ*S*exp(-q*T)*norm_cdf(typ*d1)-typ*K*exp(-r*T)*norm_cdf(typ*d2)
    return price

S = 100.0
K = 100.0
T = 5.0
vol = 0.25
r = 0.02
q = 0.03
typ = +1

In [4]:
%%cython -a

cimport cython
from libc.math cimport exp, sqrt, pow, log, erf

@cython.cdivision(True)
cdef double norm_cdf(double x):
    return 0.5*(1.0+erf(x/sqrt(2.0)))

@cython.cdivision(True)
def BS_price_C(double S, double K, double T, double vol, double r, double q, double typ):
    """return BS price for call/put for typ +1/-1"""
    cdef double d1, d2, price
    d1 = (log(S/K)+(r-q+0.5*pow(vol, 2))*T)/(vol*sqrt(T))
    d2 = d1-vol*sqrt(T)
    price = typ*S*exp(-q*T)*norm_cdf(typ*d1)-typ*K*exp(-r*T)*norm_cdf(typ*d2)
    return price

In [5]:
print BS_price_1(S, K, T, vol, r, q, typ)
print BS_price_2(S, K, T, vol, r, q, typ)
print BS_price_C(S, K, T, vol, r, q, typ)

17.3030230215
17.3030230215
17.3030230215


In [6]:
%timeit BS_price_1(S, K, T, vol, r, q, typ)
%timeit BS_price_2(S, K, T, vol, r, q, typ)
%timeit BS_price_C(S, K, T, vol, r, q, typ)

10000 loops, best of 3: 63.7 µs per loop
The slowest run took 8.47 times longer than the fastest. This could mean that an intermediate result is being cached 
1000000 loops, best of 3: 1.77 µs per loop
The slowest run took 9.93 times longer than the fastest. This could mean that an intermediate result is being cached 
1000000 loops, best of 3: 216 ns per loop
